<a href="https://colab.research.google.com/github/sebi061/VideoAdEngagement/blob/main/2_Training_feature%20extraction%20models/1_1_Intentonomy_dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Imports ###
###############

import numpy as np
import pandas as pd
import json
import os
import cv2
from urllib.request import urlopen
from urllib.error import HTTPError
import statistics as stat

import shutil

In [ ]:
### Set data directory
##################

# connect to drive
from google.colab import drive
drive.mount('/content/drive')

# set data directory
data_dir = '/content/drive/MyDrive/VideoAdEngagement/2_Training_feature extraction models/Data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Prepare Train Data and save

In [ ]:
### Import Data ###
###################

data_path = os.path.join(data_dir, 'intentonomy_train2020.json')
with open(data_path) as json_data:
    json_df_train = json.load(json_data)

In [ ]:
### The json data consist of the following components ###
#########################################################

print(json_df_train['info'])
print(json_df_train['images'][0])
print(json_df_train['categories'])
print(json_df_train['annotations'][0])

{'year': 2020, 'version': 2, 'description': 'Intentonomy training data', 'date_created': '2020-03-02 11:03:14'}
{'id': 'debb7536bac60975e92f0d1e71e9f8ab', 'filename': 'low/debb7536bac60975e92f0d1e71e9f8ab.jpg', 'original_id': 'aeVA-j1y2BY', 'unsplash_url': 'https://images.unsplash.com/4/cuba-cup.jpg?ixid=eyJhcHBfaWQiOi0xMywib3BlbnNvdXJjZSI6dHJ1ZSwidHJhY2tpbmciOmZhbHNlfQ==&q=30'}
[{'id': 0, 'name': 'Attractive'}, {'id': 1, 'name': 'BeatCompete'}, {'id': 2, 'name': 'Communicate'}, {'id': 3, 'name': 'CreativeUnique'}, {'id': 4, 'name': 'CuriousAdventurousExcitingLife'}, {'id': 5, 'name': 'EasyLife'}, {'id': 6, 'name': 'EnjoyLife'}, {'id': 7, 'name': 'FineDesignLearnArt-Arch'}, {'id': 8, 'name': 'FineDesignLearnArt-Art'}, {'id': 9, 'name': 'FineDesignLearnArt-Culture'}, {'id': 10, 'name': 'GoodParentEmoCloseChild'}, {'id': 11, 'name': 'Happy'}, {'id': 12, 'name': 'HardWorking'}, {'id': 13, 'name': 'Harmony'}, {'id': 14, 'name': 'Health'}, {'id': 15, 'name': 'InLove'}, {'id': 16, 'name': 'I

In [ ]:
### Bring into pandas formats ###
#################################

# prepare image df
img_train = pd.DataFrame(json_df_train['images'])
img_train.rename(columns={"id": "image_id"}, inplace = True)
img_train

,image_id,filename,original_id,unsplash_url
0,debb7536bac60975e92f0d1e71e9f8ab,low/debb7536bac60975e92f0d1e71e9f8ab.jpg,aeVA-j1y2BY,https://images.unsplash.com/4/cuba-cup.jpg?ixi...
1,8dea967cfe280776236e2f0d30d65349,low/8dea967cfe280776236e2f0d30d65349.jpg,d19by2PLaPc,https://images.unsplash.com/6/mountain.JPG?ixi...
2,1d994b8c044afc11012c613e67dc2705,low/1d994b8c044afc11012c613e67dc2705.jpg,AHBiSKaENwc,https://images.unsplash.com/7/girl-flowers.jpg...
3,2befcf0f6f0872bbbab96d8da8dddef4,low/2befcf0f6f0872bbbab96d8da8dddef4.jpg,2FrX56QL7P8,https://images.unsplash.com/7/sunset-hair.jpg?...
4,21e31c5697b538f6f9a6c177b52165a1,low/21e31c5697b538f6f9a6c177b52165a1.jpg,DwTZwZYi9Ww,https://images.unsplash.com/11/converse-fields...
...,...,...,...,...
12735,fc67cae19b1466f516075ba1ef3f68a2,low/fc67cae19b1466f516075ba1ef3f68a2.jpg,fniDT9K7rkw,https://images.unsplash.com/photo-1558259299-5...
12736,2770090f908d005511f534a941f25c49,low/2770090f908d005511f534a941f25c49.jpg,i3PakNjFfAc,https://images.unsplash.com/photo-1558276561-9...
12737,9ab75b96fd12daf0eb25adc197110748,low/9ab75b96fd12daf0eb25adc197110748.jpg,dkOQO29m71M,https://images.unsplash.com/photo-1558276957-a...
12738,d055ddb9238455bb3da709d0d66d91eb,low/d055ddb9238455bb3da709d0d66d91eb.jpg,3iclkJv9EGQ,https://images.unsplash.com/photo-1558305596-a...


In [ ]:
# prepare annotation df
ann_train = pd.DataFrame(json_df_train['annotations'])
ann_train.drop('id', axis = 1, inplace=True)
ann_train

,image_id,category_ids_softprob
0,3d518a3cb638ab0f39e82b4d155931be,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,980a1c6be91b123036205ece4520ac58,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,c605f8c10a344c3754463ff9267dcaf1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,e36faf2712adc57220ac0b42042a903f,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,43cdb4fc9f11d149c5e6aec015e45995,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
12735,e4decccc760328d6cd75446afb12aec5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
12736,78a7e3aed01df27e23d1d64a84146096,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
12737,b0540643c9cc392f4f17a95ea0baa5f4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
12738,278d2acf5e8a586178d7336db38106c7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
# merge
df_train = pd.merge(img_train, ann_train, on = 'image_id', how = 'inner')
df_train.head()

,image_id,filename,original_id,unsplash_url,category_ids_softprob
0,debb7536bac60975e92f0d1e71e9f8ab,low/debb7536bac60975e92f0d1e71e9f8ab.jpg,aeVA-j1y2BY,https://images.unsplash.com/4/cuba-cup.jpg?ixi...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,8dea967cfe280776236e2f0d30d65349,low/8dea967cfe280776236e2f0d30d65349.jpg,d19by2PLaPc,https://images.unsplash.com/6/mountain.JPG?ixi...,"[0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, ..."
2,1d994b8c044afc11012c613e67dc2705,low/1d994b8c044afc11012c613e67dc2705.jpg,AHBiSKaENwc,https://images.unsplash.com/7/girl-flowers.jpg...,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.333..."
3,2befcf0f6f0872bbbab96d8da8dddef4,low/2befcf0f6f0872bbbab96d8da8dddef4.jpg,2FrX56QL7P8,https://images.unsplash.com/7/sunset-hair.jpg?...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,21e31c5697b538f6f9a6c177b52165a1,low/21e31c5697b538f6f9a6c177b52165a1.jpg,DwTZwZYi9Ww,https://images.unsplash.com/11/converse-fields...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6666666666666..."


In [ ]:
### Prepare train dataset ###
#############################

# mapping dict

intent_cat =   {0: 'security and belonging',
                1: 'power',
                2: 'security and belonging',
                3: 'openness to experience',
                4: 'openness to experience',
                5: 'financial and occupational success',
                6: 'openness to experience',
                7: 'openness to experience',
                8: 'openness to experience',
                9: 'openness to experience',
                10: 'familiy',
                11: 'self-fulfill',
                12: 'ambition and ability',
                13: 'self-fulfill',
                14: 'health',
                15: 'security and belonging',
                16: 'security and belonging',
                17: 'virtue',
                18: 'ambition and ability',
                19: 'openness to experience',
                20: 'openness to experience',
                21: 'openness to experience',
                22: 'security and belonging',
                23: 'security and belonging',
                24: 'financial and occupational success',
                25: 'virtue',
                26: 'ambition and ability',
                27: 'financial and occupational success'
                }

In [ ]:
# define function for assignment
def assign_cat(l):

  count_dict = dict()

  for i, soft_prob in enumerate(l):
    count_dict[intent_cat[i]] = count_dict.get(intent_cat[i], 0) + soft_prob

  return max(count_dict, key=count_dict.get)

In [ ]:
# assign categories and save to new column in df
df_train['category'] = df_train['category_ids_softprob'].apply(assign_cat)

In [ ]:
# check distribution
df_train['category'].value_counts()

openness to experience                6104
security and belonging                3756
self-fulfill                           715
ambition and ability                   658
financial and occupational success     576
familiy                                306
power                                  300
virtue                                 168
health                                 157
Name: category, dtype: int64

In [ ]:
# take subset to get balanced distribution of counts for labels
df_train_subset = df_train.groupby('category').sample(n = 100, random_state = 42).reset_index(drop = True)

In [ ]:
# check distribution
df_train_subset['category'].value_counts()

ambition and ability                  100
familiy                               100
financial and occupational success    100
health                                100
openness to experience                100
power                                 100
security and belonging                100
self-fulfill                          100
virtue                                100
Name: category, dtype: int64

In [ ]:
### Download pictures ###
#########################

# make folder for saving
os.makedirs('Intentonomy_train')

# set path to folder
SAVE_PATH = './Intentonomy_train'

missing_urls = []

for index, row in df_train_subset.iterrows():

  # Load image from url as numpy array
  try:
    image = np.asarray(bytearray(urlopen(row['unsplash_url']).read()), dtype="uint8")
  except HTTPError:
    missing_urls.append(row['unsplash_url'])
    continue


  image = np.asarray(bytearray(urlopen(row['unsplash_url']).read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)

  # Scale image down in size
  image = cv2.pyrDown(cv2.pyrDown(cv2.pyrDown(image)))


  # get filename
  file_name = os.path.join(SAVE_PATH, row['image_id']) + '.jpeg'

  # save image
  cv2.imwrite(file_name, image)

In [ ]:
### Zip files and save to gdrive ###
####################################

# zip
shutil.make_archive("./Intentonomy_train", 'zip', SAVE_PATH)

'/content/Intentonomy_train.zip'

In [ ]:
# save to gdrive
shutil.copy('./Intentonomy_train.zip', data_dir)

'/content/drive/MyDrive/0_Masterarbeit/4_High_Level_Features/data/Intentonomy_train.zip'

In [ ]:
### Save dataframe to gdrive as csv ###
#######################################

# check missing urls
missing_urls

['https://images.unsplash.com/photo-1497171156029-51dfc973e5f9?ixid=eyJhcHBfaWQiOi0xMywib3BlbnNvdXJjZSI6dHJ1ZSwidHJhY2tpbmciOmZhbHNlfQ==&q=30',
 'https://images.unsplash.com/photo-1482245294234-b3f2f8d5f1a4?ixid=eyJhcHBfaWQiOi0xMywib3BlbnNvdXJjZSI6dHJ1ZSwidHJhY2tpbmciOmZhbHNlfQ==&q=30',
 'https://images.unsplash.com/uploads/14132232941229e1b3002/86744abf?ixid=eyJhcHBfaWQiOi0xMywib3BlbnNvdXJjZSI6dHJ1ZSwidHJhY2tpbmciOmZhbHNlfQ==&q=30']

In [ ]:
# remove missing urls before saving
df_train_subset = df_train_subset[~df_train_subset['unsplash_url'].isin(missing_urls)]

In [ ]:
# check if 2 are removed
df_train_subset['category'].value_counts()

familiy                               100
financial and occupational success    100
health                                100
openness to experience                100
power                                 100
security and belonging                100
self-fulfill                          100
virtue                                 99
ambition and ability                   98
Name: category, dtype: int64

In [ ]:
# save as csv
df_train_subset.to_csv('./Intentonomy_train_subset.csv')

In [ ]:
# save to gdrive
shutil.copy('./Intentonomy_train_subset.csv', data_dir)

'/content/drive/MyDrive/0_Masterarbeit/4_High_Level_Features/data/Intentonomy_train_subset.csv'

### Prepare and save test dataset

In [ ]:
### Import Data ###
###################

# json file

data_path_test = os.path.join(data_dir, 'intentonomy_val2020.json')

with open(data_path_test) as json_data:
    json_df_test = json.load(json_data)

In [ ]:
### Bring into pandas formats ###
#################################

# prepare image df
img_test = pd.DataFrame(json_df_test ['images'])
img_test.rename(columns={"id": "image_id"}, inplace = True)
img_test

,image_id,filename,original_id,unsplash_url
0,ce2b1c756b60afaca18fff022212d3f4,low/ce2b1c756b60afaca18fff022212d3f4.jpg,X7SkpHKEQQs,https://images.unsplash.com/photo-145313466567...
1,41206cd6b4457a898bc69d59e32e172d,low/41206cd6b4457a898bc69d59e32e172d.jpg,FTsyHMwg_UM,https://images.unsplash.com/photo-149409495716...
2,88953ebdeda4e4b89ab675fea299023f,low/88953ebdeda4e4b89ab675fea299023f.jpg,l_YNobbDYJk,https://images.unsplash.com/photo-149727167942...
3,037d365f9757266c05e84a164e342930,low/037d365f9757266c05e84a164e342930.jpg,TLBplYQvqn0,https://images.unsplash.com/photo-151772884877...
4,8a4f01b8074e6f2dc947d820f895e585,low/8a4f01b8074e6f2dc947d820f895e585.jpg,QiqHfDDf8B4,https://images.unsplash.com/photo-1548884481-d...
...,...,...,...,...
493,29c235e2cba5b14eabd6a14c55df5eae,low/29c235e2cba5b14eabd6a14c55df5eae.jpg,d9U7yv2Uhzc,https://images.unsplash.com/photo-1561175251-2...
494,2c614ffdcb4602858c32bdc915281459,low/2c614ffdcb4602858c32bdc915281459.jpg,NWe_nlC0tGQ,https://images.unsplash.com/photo-150496275942...
495,1f459cb08b6d68bd5c812b3f54f354fb,low/1f459cb08b6d68bd5c812b3f54f354fb.jpg,csHw20wsqfs,https://images.unsplash.com/photo-148109309677...
496,e9e7cf98351be19479dc03ffe8295504,low/e9e7cf98351be19479dc03ffe8295504.jpg,Ap3fFS0iOiE,https://images.unsplash.com/photo-1543058871-7...


In [ ]:
# prepare annotation df
ann_test = pd.DataFrame(json_df_test['annotations'])
ann_test.drop('id', axis = 1, inplace=True)
ann_test

,image_id,category_ids
0,ce2b1c756b60afaca18fff022212d3f4,"[6, 19]"
1,41206cd6b4457a898bc69d59e32e172d,"[4, 9, 12]"
2,88953ebdeda4e4b89ab675fea299023f,[9]
3,037d365f9757266c05e84a164e342930,[11]
4,8a4f01b8074e6f2dc947d820f895e585,"[0, 11]"
...,...,...
493,29c235e2cba5b14eabd6a14c55df5eae,"[0, 3, 11]"
494,2c614ffdcb4602858c32bdc915281459,[21]
495,1f459cb08b6d68bd5c812b3f54f354fb,"[0, 6, 11, 15, 19]"
496,e9e7cf98351be19479dc03ffe8295504,"[8, 11]"


In [ ]:
# merge
df_test = pd.merge(img_test, ann_test, on = 'image_id', how = 'inner')
df_test.head()

,image_id,filename,original_id,unsplash_url,category_ids
0,ce2b1c756b60afaca18fff022212d3f4,low/ce2b1c756b60afaca18fff022212d3f4.jpg,X7SkpHKEQQs,https://images.unsplash.com/photo-145313466567...,"[6, 19]"
1,41206cd6b4457a898bc69d59e32e172d,low/41206cd6b4457a898bc69d59e32e172d.jpg,FTsyHMwg_UM,https://images.unsplash.com/photo-149409495716...,"[4, 9, 12]"
2,88953ebdeda4e4b89ab675fea299023f,low/88953ebdeda4e4b89ab675fea299023f.jpg,l_YNobbDYJk,https://images.unsplash.com/photo-149727167942...,[9]
3,037d365f9757266c05e84a164e342930,low/037d365f9757266c05e84a164e342930.jpg,TLBplYQvqn0,https://images.unsplash.com/photo-151772884877...,[11]
4,8a4f01b8074e6f2dc947d820f895e585,low/8a4f01b8074e6f2dc947d820f895e585.jpg,QiqHfDDf8B4,https://images.unsplash.com/photo-1548884481-d...,"[0, 11]"


In [ ]:
# define function for assignment
def assign_cat(l):
  cat = []
  for i in l:
    cat.append(intent_cat[i])

  return list(set(cat))

In [ ]:
# assign categories and save to new column in df
df_test['category'] = df_test['category_ids'].apply(assign_cat)

In [ ]:
# check
df_test

,image_id,filename,original_id,unsplash_url,category_ids,category
0,ce2b1c756b60afaca18fff022212d3f4,low/ce2b1c756b60afaca18fff022212d3f4.jpg,X7SkpHKEQQs,https://images.unsplash.com/photo-145313466567...,"[6, 19]",[openness to experience]
1,41206cd6b4457a898bc69d59e32e172d,low/41206cd6b4457a898bc69d59e32e172d.jpg,FTsyHMwg_UM,https://images.unsplash.com/photo-149409495716...,"[4, 9, 12]","[ambition and ability, openness to experience]"
2,88953ebdeda4e4b89ab675fea299023f,low/88953ebdeda4e4b89ab675fea299023f.jpg,l_YNobbDYJk,https://images.unsplash.com/photo-149727167942...,[9],[openness to experience]
3,037d365f9757266c05e84a164e342930,low/037d365f9757266c05e84a164e342930.jpg,TLBplYQvqn0,https://images.unsplash.com/photo-151772884877...,[11],[self-fulfill]
4,8a4f01b8074e6f2dc947d820f895e585,low/8a4f01b8074e6f2dc947d820f895e585.jpg,QiqHfDDf8B4,https://images.unsplash.com/photo-1548884481-d...,"[0, 11]","[security and belonging, self-fulfill]"
...,...,...,...,...,...,...
493,29c235e2cba5b14eabd6a14c55df5eae,low/29c235e2cba5b14eabd6a14c55df5eae.jpg,d9U7yv2Uhzc,https://images.unsplash.com/photo-1561175251-2...,"[0, 3, 11]","[security and belonging, self-fulfill, opennes..."
494,2c614ffdcb4602858c32bdc915281459,low/2c614ffdcb4602858c32bdc915281459.jpg,NWe_nlC0tGQ,https://images.unsplash.com/photo-150496275942...,[21],[openness to experience]
495,1f459cb08b6d68bd5c812b3f54f354fb,low/1f459cb08b6d68bd5c812b3f54f354fb.jpg,csHw20wsqfs,https://images.unsplash.com/photo-148109309677...,"[0, 6, 11, 15, 19]","[security and belonging, self-fulfill, opennes..."
496,e9e7cf98351be19479dc03ffe8295504,low/e9e7cf98351be19479dc03ffe8295504.jpg,Ap3fFS0iOiE,https://images.unsplash.com/photo-1543058871-7...,"[8, 11]","[self-fulfill, openness to experience]"


In [ ]:
### Download pictures ###
#########################

# make folder for saving
os.makedirs('Intentonomy_test')

# set path to folder
SAVE_PATH = './Intentonomy_test'

missing_urls = []

for index, row in df_test.iterrows():

  # Load image from url as numpy array
  try:
    image = np.asarray(bytearray(urlopen(row['unsplash_url']).read()), dtype="uint8")
  except HTTPError:
    missing_urls.append(row['unsplash_url'])
    continue


  image = np.asarray(bytearray(urlopen(row['unsplash_url']).read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)

  # Scale image down in size
  image = cv2.pyrDown(cv2.pyrDown(cv2.pyrDown(image)))


  # get filename
  file_name = os.path.join(SAVE_PATH, row['image_id']) + '.jpeg'

  # save image
  cv2.imwrite(file_name, image)

In [ ]:
### Zip files and save to gdrive ###
####################################

# zip
shutil.make_archive("./Intentonomy_test", 'zip', SAVE_PATH)

'/content/Intentonomy_test.zip'

In [ ]:
# save to gdrive
shutil.copy('./Intentonomy_test.zip', data_dir)

'/content/drive/MyDrive/0_Masterarbeit/4_High_Level_Features/data/Intentonomy_test.zip'

In [ ]:
### Save dataframe to gdrive as csv ###
#######################################

# check missing urls
missing_urls

[]

In [ ]:
# save as csv
df_test.to_csv('./Intentonomy_test.csv')

In [ ]:
# save to gdrive
shutil.copy('./Intentonomy_test.csv', data_dir)

'/content/drive/MyDrive/0_Masterarbeit/4_High_Level_Features/data/Intentonomy_test.csv'